# Data Validation with Pydantic: Base Model Extension

## Generic Example

In [ ]:
from sqlmodel import SQLModel, Field
from pydantic import create_model
from typing import Optional


def validate_with_pydantic(model):
    annotations = model.__annotations__
    kwargs = model.__dict__
    pydantic_annotations = {}
    for field, ann in annotations.items():
        if ann.__module__ == "typing":
            if getattr(ann, "__origin__", None) is None and type(None) in ann.__args__:
                pydantic_annotations[field] = (ann, None)
        else:
            pydantic_annotations[field] = (ann, ...)
    validation_model = create_model(model.__class__.__name__, **pydantic_annotations)
    validation_model.validate(kwargs)
    return


class SQLModelBase(SQLModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        validate_with_pydantic(self)


class Car(SQLModelBase, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    year: int

In [ ]:
# Data validation works as expected (error raised for missing field)
car = Car()

In [ ]:
# Data validation works as expected (no errors raised)
car = Car(name="Test Car", year=1995)
car

## Lamin Example

In [ ]:
import lamindb.schema as lns

In [ ]:
??lns.dev.sqlmodel.SQLModelModule

In [ ]:
??lns.dev.sqlmodel.SQLModelPrefix

In [ ]:
# Data validation works as expected (error raised for missing field)
pipeline = lns.Pipeline()

In [ ]:
# Data validation works as expected (no errors raised)

from datetime import datetime

pipeline = lns.Pipeline(created_at=datetime.now())
pipeline

In [ ]:
# Data validation does not work for complex type (familiar "ForwardRef" error)
run = lns.Run(name="Test Run", pipeline=pipeline)
run